In [1]:
import os

SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'

os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark


from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]")\
                            .appName("SparkStreamingKafkaBasic").getOrCreate()

22/04/01 21:19:01 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/04/01 21:19:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fa882ff3-8944-42e6-a848-b0a49cdb62c2;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 711ms :: artifacts dl 14ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [4]:
kafkaDf = spark.readStream.format("kafka")\
              .option("kafka.bootstrap.servers", "localhost:9092")\
              .option("subscribe", "orders")\
              .load()

In [5]:
kafkaDf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
orderRawDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
orderRawDf.printSchema()

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [7]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType, DateType, LongType

schema = StructType([
            StructField("order_id", IntegerType(), True),
            StructField("item_id", StringType(), True),
            StructField("price", IntegerType(), True),
            StructField("qty", IntegerType(), True),
            StructField("order_date", LongType(), True),
            StructField("state", StringType(), True),
        ])

In [8]:
jsonDf = orderRawDf.withColumn("value", F.from_json("value", schema))
jsonDf.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- order_id: integer (nullable = true)
 |    |-- item_id: string (nullable = true)
 |    |-- price: integer (nullable = true)
 |    |-- qty: integer (nullable = true)
 |    |-- order_date: long (nullable = true)
 |    |-- state: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [9]:
orderDf = jsonDf.select(F.col("value.*"))
orderDf.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- item_id: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- qty: integer (nullable = true)
 |-- order_date: long (nullable = true)
 |-- state: string (nullable = true)



In [10]:
orderDf = orderDf\
                .withColumn("timestampTemp", (F.col("order_date") / 1000).cast("timestamp"))\
                .withColumn("order_time", F.date_trunc("minute", F.col("timestampTemp")))\
                .drop("order_date")\
                .drop("timestampTemp")\
                .withColumnRenamed("order_time", "timestamp")

orderDf.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- item_id: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- qty: integer (nullable = true)
 |-- state: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [11]:
orderDf5min= orderDf.withColumn("amount", (F.col("price") * F.col("qty")))\
                    .withWatermark("timestamp", "1 minutes")\
                    .groupBy("state", F.window("timestamp", "5 minutes"))\
                    .agg( F.sum("amount").alias("amount"))

orderDf5min.printSchema() 

root
 |-- state: string (nullable = true)
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- amount: long (nullable = true)



In [12]:
# echoOnconsole = orderDf5min\
#                  .writeStream\
#                 .outputMode("update")\
#                 .format("console")\
#                 .option("truncate", False)\
#                 .start() # start the query. spark will subscribe for data

In [13]:
orderDf5minKafka = orderDf5min\
                            .selectExpr("to_json(struct(*)) AS value")

orderDf5minKafka\
            .writeStream\
             .format("kafka")\
            .outputMode("update")\
             .option("kafka.bootstrap.servers", "localhost:9092")\
            .option("topic", "statewise-earning")\
            .option("checkpointLocation", "file:///tmp/spark31")\
            .start()

22/04/01 21:20:34 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-28e1d48c-43fd-46e9-a6fa-2da364180da0-1811380348-driver-0-1, groupId=spark-kafka-source-28e1d48c-43fd-46e9-a6fa-2da364180da0-1811380348-driver-0] Error while fetching metadata with correlation id 2 : {orders=LEADER_NOT_AVAILABLE}


In [14]:
def processBatchData(candleBatchDf, batch_id):
    print ("process batch called", batch_id, "writing ", candleBatchDf.count())
    
    (
     candleBatchDf
        .select('*', F.col("window.*"))
        .withColumnRenamed("start", "start_time")
        .withColumnRenamed("end", "end_time")
        .drop("window")
        .write
        .format("jdbc")
        .mode("append")
        .option("url", "jdbc:mysql://localhost:3306/stockdb?allowPublicKeyRetrieval=true&useSSL=false")
        .option("driver", "com.mysql.jdbc.Driver")
        .option("user", "team")
        .option("password", "Team1234!")
        .option("dbtable", "StatewiseEarning")
         .save()
    )
    
orderDf5min.writeStream.outputMode("append").foreachBatch(processBatchData).start()

22/04/01 21:37:12 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d7b07586-a31f-4328-9c36-a4bc1aae90e0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


process batch called 0 writing  0
